# Análise Exploratória de Dados (EDA)
## Jigsaw Toxic Comment Classification Challenge

Este notebook realiza uma análise exploratória completa dos dados da competição.

## 1. Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Bibliotecas importadas com sucesso!")

## 2. Carregamento dos Dados

In [ ]:
# Definir caminhos
data_dir = Path('../data/raw')

# Carregar dados de treino
print("Carregando train.csv...")
train_df = pd.read_csv(data_dir / 'train.csv')
print(f"Shape: {train_df.shape}")

# Carregar dados de teste
print("\nCarregando test.csv...")
test_df = pd.read_csv(data_dir / 'test.csv')
print(f"Shape: {test_df.shape}")

# Carregar sample submission
print("\nCarregando sample_submission.csv...")
sample_submission = pd.read_csv(data_dir / 'sample_submission.csv')
print(f"Shape: {sample_submission.shape}")

## 3. Primeira Visualização dos Dados

In [ ]:
print("=== DADOS DE TREINO ===")
print(f"\nPrimeiras linhas:")
display(train_df.head())

print(f"\nInformações sobre o dataset:")
print(train_df.info())

print(f"\nEstatísticas descritivas:")
display(train_df.describe())

In [ ]:
print("=== DADOS DE TESTE ===")
print(f"\nPrimeiras linhas:")
display(test_df.head())

print(f"\nInformações sobre o dataset:")
print(test_df.info())

## 4. Análise das Colunas de Toxicidade

In [ ]:
# Identificar colunas de toxicidade
toxic_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

print("Colunas de toxicidade:", toxic_cols)
print(f"\nTotal de colunas no dataset: {len(train_df.columns)}")
print(f"Colunas: {list(train_df.columns)}")

In [ ]:
# Estatísticas das colunas de toxicidade
print("=== ESTATÍSTICAS DAS CLASSES DE TOXICIDADE ===")
toxic_stats = train_df[toxic_cols].describe()
display(toxic_stats)

print("\n=== CONTAGEM DE COMENTÁRIOS POR CLASSE ===")
for col in toxic_cols:
    count = train_df[col].sum()
    pct = (count / len(train_df)) * 100
    print(f"{col:20s}: {count:6d} ({pct:5.2f}%)")

## 5. Visualizações - Distribuição das Classes

In [ ]:
# Gráfico de barras com a contagem de cada classe
fig, ax = plt.subplots(figsize=(12, 6))

counts = train_df[toxic_cols].sum().sort_values(ascending=False)
counts.plot(kind='bar', ax=ax, color='coral')
ax.set_title('Distribuição das Classes de Toxicidade', fontsize=14, fontweight='bold')
ax.set_xlabel('Tipo de Toxicidade', fontsize=12)
ax.set_ylabel('Número de Comentários', fontsize=12)
ax.tick_params(axis='x', rotation=45)

# Adicionar valores nas barras
for i, v in enumerate(counts):
    ax.text(i, v + 1000, f'{int(v):,}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de porcentagem
fig, ax = plt.subplots(figsize=(12, 6))

percentages = (train_df[toxic_cols].sum() / len(train_df) * 100).sort_values(ascending=False)
percentages.plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Percentual de Comentários por Classe de Toxicidade', fontsize=14, fontweight='bold')
ax.set_xlabel('Tipo de Toxicidade', fontsize=12)
ax.set_ylabel('Percentual (%)', fontsize=12)
ax.tick_params(axis='x', rotation=45)

# Adicionar valores nas barras
for i, v in enumerate(percentages):
    ax.text(i, v + 0.5, f'{v:.2f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 6. Análise de Comentários Múltiplas Classes

In [ ]:
# Contar quantas classes cada comentário tem
train_df['num_classes'] = train_df[toxic_cols].sum(axis=1)

print("=== DISTRIBUIÇÃO DO NÚMERO DE CLASSES POR COMENTÁRIO ===")
class_distribution = train_df['num_classes'].value_counts().sort_index()
print(class_distribution)

print(f"\nComentários sem toxicidade (0 classes): {class_distribution.get(0, 0):,}")
print(f"Comentários com pelo menos 1 classe: {(train_df['num_classes'] > 0).sum():,}")
print(f"Comentários com múltiplas classes (2+): {(train_df['num_classes'] >= 2).sum():,}")

In [ ]:
# Visualização da distribuição
fig, ax = plt.subplots(figsize=(10, 6))

class_distribution.plot(kind='bar', ax=ax, color='mediumseagreen')
ax.set_title('Distribuição do Número de Classes por Comentário', fontsize=14, fontweight='bold')
ax.set_xlabel('Número de Classes', fontsize=12)
ax.set_ylabel('Número de Comentários', fontsize=12)

# Adicionar valores nas barras
for i, v in enumerate(class_distribution):
    ax.text(i, v + 5000, f'{int(v):,}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 7. Análise de Correlação entre Classes

In [ ]:
# Matriz de correlação
correlation_matrix = train_df[toxic_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlação entre Classes de Toxicidade', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Análise do Texto dos Comentários

In [ ]:
# Estatísticas básicas sobre o texto
train_df['comment_length'] = train_df['comment_text'].str.len()
train_df['word_count'] = train_df['comment_text'].str.split().str.len()

print("=== ESTATÍSTICAS DO TEXTO ===")
print(f"Comprimento médio dos comentários: {train_df['comment_length'].mean():.0f} caracteres")
print(f"Comprimento mediano dos comentários: {train_df['comment_length'].median():.0f} caracteres")
print(f"Número médio de palavras: {train_df['word_count'].mean():.1f} palavras")
print(f"Número mediano de palavras: {train_df['word_count'].median():.1f} palavras")
print(f"\nComentário mais curto: {train_df['comment_length'].min()} caracteres")
print(f"Comentário mais longo: {train_df['comment_length'].max()} caracteres")

In [ ]:
# Distribuição do comprimento dos comentários
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma do comprimento
axes[0].hist(train_df['comment_length'], bins=50, color='skyblue', edgecolor='black')
axes[0].set_title('Distribuição do Comprimento dos Comentários', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Número de Caracteres', fontsize=10)
axes[0].set_ylabel('Frequência', fontsize=10)
axes[0].axvline(train_df['comment_length'].mean(), color='red', linestyle='--', 
                label=f'Média: {train_df["comment_length"].mean():.0f}')
axes[0].legend()

# Histograma do número de palavras
axes[1].hist(train_df['word_count'], bins=50, color='lightcoral', edgecolor='black')
axes[1].set_title('Distribuição do Número de Palavras', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Número de Palavras', fontsize=10)
axes[1].set_ylabel('Frequência', fontsize=10)
axes[1].axvline(train_df['word_count'].mean(), color='red', linestyle='--', 
                label=f'Média: {train_df["word_count"].mean():.1f}')
axes[1].legend()

plt.tight_layout()
plt.show()

## 9. Exemplos de Comentários

In [ ]:
# Exemplos de comentários não tóxicos
print("=== EXEMPLOS DE COMENTÁRIOS NÃO TÓXICOS ===")
non_toxic = train_df[train_df['num_classes'] == 0]
for idx, row in non_toxic.head(3).iterrows():
    print(f"\nID: {row['id']}")
    print(f"Texto: {row['comment_text'][:200]}...")
    print("-" * 80)

In [ ]:
# Exemplos de comentários tóxicos
print("=== EXEMPLOS DE COMENTÁRIOS TÓXICOS ===")
toxic = train_df[train_df['num_classes'] > 0]
for idx, row in toxic.head(3).iterrows():
    print(f"\nID: {row['id']}")
    classes = [col for col in toxic_cols if row[col] == 1]
    print(f"Classes: {', '.join(classes)}")
    print(f"Texto: {row['comment_text'][:200]}...")
    print("-" * 80)

## 10. Resumo e Conclusões

In [ ]:
print("=== RESUMO DA ANÁLISE EXPLORATÓRIA ===")
print(f"\n1. Dataset de Treino:")
print(f"   - Total de comentários: {len(train_df):,}")
print(f"   - Comentários não tóxicos: {(train_df['num_classes'] == 0).sum():,} ({(train_df['num_classes'] == 0).sum()/len(train_df)*100:.2f}%)")
print(f"   - Comentários tóxicos: {(train_df['num_classes'] > 0).sum():,} ({(train_df['num_classes'] > 0).sum()/len(train_df)*100:.2f}%)")

print(f"\n2. Distribuição das Classes:")
for col in toxic_cols:
    count = train_df[col].sum()
    pct = (count / len(train_df)) * 100
    print(f"   - {col:20s}: {count:6,} ({pct:5.2f}%)")

print(f"\n3. Características do Texto:")
print(f"   - Comprimento médio: {train_df['comment_length'].mean():.0f} caracteres")
print(f"   - Palavras médias: {train_df['word_count'].mean():.1f} palavras")

print(f"\n4. Dataset de Teste:")
print(f"   - Total de comentários: {len(test_df):,}")

print("\n=== PRÓXIMOS PASSOS ===")
print("1. Pré-processamento de texto")
print("2. Feature engineering")
print("3. Treinamento de modelos")
print("4. Avaliação e otimização")